In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX
from csv import writer

In [2]:
#Change working directory to navicage files
os.chdir("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")

In [3]:
#Function to extract 1045 sheet name from xlsx files

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file, data_only=True)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [4]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(file):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        try: #when sheet is nonetype (doesn't exist) we need to skip it
            for row in sheet.iter_rows():
                    for cell in row:
                        heading = str(cell.value).lower()
                        wordsInHeading=heading.split()
                        match =  all(item in wordsInHeading for item in wordlist)
                        if match==True:
                            coord1=cell.coordinate
                            coordlist.append(coord1)
                            break
        except:
            return("FAIL")
            continue
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [5]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [6]:
#Function to extract all dataframes from a single 1095 file
def getMaster(somefile):
        sheet=find1045_xlsx(somefile)
        masterlist=[]
        rawlist=getdata_xlsx(somefile)
        for i in range(len(rawlist)):
                df=rawlist[i].applymap(convertvalue)
                raw_name=sheet["A1"].value
                hos_name=raw_name.replace('Hospital Name: ', '')
                df["Hospital"] = os.path.basename(os.path.dirname(somefile))
                df.columns = df.iloc[0]
                df["Service Type"]=df.iloc[0,0]
                df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
                df = df.iloc[1: , :]
                        #add the hospital name as a variable
                        
                masterlist.append(df)
                return(masterlist)

In [7]:
#Function to get a list of files in a directory (code borrowed from StackOverflow)

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries99213/G0463
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [8]:
#Create Column Labels
alldata=pd.DataFrame(columns = ['ID','Service', 'CPT',"AvCharge", 'Hospital', "ServiceType"])
with open('/home/ygael/Projects/healthcare/data/csv/alldata.csv', 'a') as f:
    alldata.to_csv(f, header=True)

In [9]:
##Extract data from as many 1045 files in the chargemaster as possible given efficacy of preceeding functions at standarizing extraction process for decidedly non-standard files

filelist=getListOfFiles("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")
for filename in filelist:
    if filename[-4:]=="xlsx":
                try:
                    df=getMaster(filename)
                    if df != "FAIL":
                        for i in range(len(df)):
                            df[i].replace({'99213/G0463': 99213, 'G0463': 99213,'-': "",  'N/A': ""}, inplace=True)
                            with open('/home/ygael/Projects/healthcare/data/csv/alldata.csv', 'a') as f:
                                    try:
                                        df[i].to_csv(f, header=False)
                                    except:
                                        continue
                except:
                    continue

/home/ygael/anaconda3/envs/DataViz/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
/home/ygael/anaconda3/envs/DataViz/lib/python3.8/site-packages/openpyxl/reader/drawings.py:59: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)


In [55]:
test=getMaster("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Dameron Hospital/106390846_CDM_All_2020.xlsx")

In [58]:
test[0].replace({'-': "",  'N/A': ""})


,Service,2020 CPT Code,Average Charge,Dameron Hospital,Service Type
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,1643,Dameron Hospital,Evaluation & Management Services (CPT Codes 99...
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,2391,Dameron Hospital,Evaluation & Management Services (CPT Codes 99...
3,"Emergency Room Visit, Level 4 (high severity w...",99284,3534,Dameron Hospital,Evaluation & Management Services (CPT Codes 99...
4,"Emergency Room Visit, Level 4 (high severity w...",99285,4887,Dameron Hospital,Evaluation & Management Services (CPT Codes 99...
5,"Outpatient Visit, established patient, 15 minutes",99213,,Dameron Hospital,Evaluation & Management Services (CPT Codes 99...


In [25]:
nulllist=["","N/A"]

,Service,CPT,Hospital,Service Type


In [66]:
csv=pd.read_csv('/home/ygael/Projects/healthcare/data/csv/alldata.csv')

In [64]:
for i in range (len(csv["CPT"])):
    if csv["CPT"][i] !="N/A":
        print(csv["CPT"][i])

KeyError: 'CPT'

In [65]:
csv

,1,"Emergency Room Visit, Level 2 (low to moderate severity)",99282,1394,Adventist Health Lodi Memorial,Evaluation & Management Services (CPT Codes 99201-99499)
0,2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1911.00,Adventist Health Lodi Memorial,Evaluation & Management Services (CPT Codes 99...
1,3,"Emergency Room Visit, Level 4 (high severity w...",99284,2560.00,Adventist Health Lodi Memorial,Evaluation & Management Services (CPT Codes 99...
2,4,"Emergency Room Visit, Level 4 (high severity w...",99285,3594.00,Adventist Health Lodi Memorial,Evaluation & Management Services (CPT Codes 99...
3,5,"Outpatient Visit, established patient, 15 minutes",99213,187.00,Adventist Health Lodi Memorial,Evaluation & Management Services (CPT Codes 99...
4,1,"Emergency Room Visit, Level 2 (low to moderate...",99282,NaN,San Joaquin Valley Rehab Center,Evaluation & Management Services (CPT Codes 99...
...,...,...,...,...,...,...
1266,5,"Outpatient Visit, established patient, 15 minutes",99213,NaN,Sierra Vista Hospital,Evaluation & Management Services (CPT Codes 99...
1267,1,"Emergency Room Visit, Level 2 (low to moderate...",99282,4019.18,Cedars-Sinai Medical Center,Evaluation & Management Services (CPT Codes 99...
1268,2,"Emergency Room Visit, Level 3 (moderate severity)",99283,6183.35,Cedars-Sinai Medical Center,Evaluation & Management Services (CPT Codes 99...
1269,3,"Emergency Room Visit, Level 4 (high severity)",99284,8833.35,Cedars-Sinai Medical Center,Evaluation & Management Services (CPT Codes 99...


In [19]:
nonull=csv.replace({'-': None, '': None,  'N/A': None})

In [31]:
csv.loc["CPT"]

KeyError: 'CPT'